In [7]:
import os

import traceback

from datetime import datetime, time
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import (StructType, StructField, IntegerType, StringType, BooleanType)
from pyspark.sql import functions as F

Define file paths


In [8]:
current_dir = os.getcwd()
file_name = '../csv_files/chapter15/orders.csv'
file_path = os.path.join(current_dir, file_name)

Data Ingestion and Do Aggregation

In [12]:
spark = SparkSession.builder.appName("Data Aggregation").getOrCreate()
df = spark.read.csv(path=file_path, header=True, inferSchema=True)
df.printSchema()

25/01/14 20:08:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/14 20:08:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/01/14 20:08:07 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/01/14 20:08:07 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/01/14 20:08:07 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/01/14 20:08:07 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/01/14 20:08:07 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/01/14 20:08:07 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
25/01/14 20:08:07 WARN Utils: Service 'SparkUI' could not bind on port 4048. Attempting port 4049.


root
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- state: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- revenue: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



Aggregation using Data Frame API

In [13]:
aggregation_df = (df.groupBy(F.col("firstName"), F.col("lastName"), F.col("state"))
                  .agg(F.sum("quantity"), F.sum("revenue"), F.avg("revenue")))
aggregation_df.show()

+------------+--------+-----+-------------+------------+------------+
|   firstName|lastName|state|sum(quantity)|sum(revenue)|avg(revenue)|
+------------+--------+-----+-------------+------------+------------+
|       Ginni| Rometty|   NY|            7|          91|        91.0|
|Jean-Georges|  Perrin|   CA|            4|          75|        75.0|
|      Holden|   Karau|   CA|           10|         190|        95.0|
|Jean-Georges|  Perrin|   NC|            3|         420|       210.0|
+------------+--------+-----+-------------+------------+------------+



Aggregation using SQL

In [15]:
df.createOrReplaceTempView("df")
aggregation_sql = """
    SELECT firstName, lastName, state, SUM(quantity), SUM(revenue), AVG(revenue)
    FROM df
    GROUP BY firstName, lastName, state
"""
aggregation_df_sql = spark.sql(aggregation_sql)
aggregation_df_sql.show()

+------------+--------+-----+-------------+------------+------------+
|   firstName|lastName|state|sum(quantity)|sum(revenue)|avg(revenue)|
+------------+--------+-----+-------------+------------+------------+
|       Ginni| Rometty|   NY|            7|          91|        91.0|
|Jean-Georges|  Perrin|   CA|            4|          75|        75.0|
|      Holden|   Karau|   CA|           10|         190|        95.0|
|Jean-Georges|  Perrin|   NC|            3|         420|       210.0|
+------------+--------+-----+-------------+------------+------------+



In [10]:
spark.stop()